In [1]:
import pandas as pd
import numpy as np
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec

from skbio import DistanceMatrix
from skbio.tree import nj
#run: pip install scikit-bio

import uts

Read Test Data

In [2]:
ddf = pd.read_csv('data/Form relationships_dummy tables.csv')
gdf_test = uts.process_data(ddf)
#gdf_test.get_group('A')

/home/sunny/miniconda3/envs/env5/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Processing Test Dataset

In [3]:
# iterate form -> sentence -> concatenat into a list [,,]
test_data = uts.df_to_sentence(gdf_test,1,0)
test_data[0]

{'A': 'Ontology Audit Id Ontology Event Id Action Action Date Action User Profile Id Field Field Name Previous Value New Value Ontology Id A'}

In [4]:
tagged_data_test= uts.tagging_data(test_data)
tagged_data_test[0]

TaggedDocument(words=['Ontology', 'Audit', 'Id', 'Ontology', 'Event', 'Id', 'Action', 'Action', 'Date', 'Action', 'User', 'Profile', 'Id', 'Field', 'Field', 'Name', 'Previous', 'Value', 'New', 'Value', 'Ontology', 'Id', 'A'], tags=['A'])

Result

Load saved model

In [5]:
loaded_model = Doc2Vec.load("data/d2v.model")

Pairwise matrix for test data

In [6]:
test_tables = pd.DataFrame(gdf_test).loc[:,0]

In [7]:
# Making table name reference
dummy_table = {}

for i, val in enumerate(test_tables):
    dummy_table[val] = i

dummy_table['A']

0

Calculate score

In [8]:
#score_ = uts.cal_score_unseen('A2', 'A4', loaded_model, test_data, dummy_table)
#score_ = uts.cal_score_cos('A2', 'A4', loaded_model, test_data, dummy_table, alpha=0.1, min_alpha=0.0001,steps=5)

Matrix for Test data

In [9]:
# dist_neg 
# 1: use negative score value 
# 0: abs(negative score value)
lst_test = uts.matrix(test_tables, loaded_model,  test_data, dummy_table, dist_neg = 1)

1.273089438676834
0.9040987268090248
1.1647024601697922
1.074260264635086
0.9834027327597141
0.9519652277231216
1.284972369670868
0.8859104365110397
1.4316421747207642
1.273089438676834
0.9485472440719604
1.1628867536783218
0.9809514451771975
0.9465484209358692
0.9468966387212276
1.033720750361681
1.0358270183205605
0.58649080991745
0.9040987268090248
0.9485472440719604
1.0342564433813095
1.0676769465208054
1.1404753178358078
1.1281826794147491
1.0571990087628365
1.00918881688267
1.210363119840622
1.1647024601697922
1.1628867536783218
1.0342564433813095
1.1186556965112686
1.0444616042077541
1.2100297659635544
1.0837761089205742
1.0235667675733566
0.8637887090444565
1.074260264635086
0.9809514451771975
1.0676769465208054
1.1186556965112686
0.9447369575500488
1.0137690640985966
0.8738696873188019
0.8673874139785767
0.9904365390539169
0.9834027327597141
0.9465484209358692
1.1404753178358078
1.0444616042077541
0.9447369575500488
1.1427073925733566
1.184932455420494
0.8471930623054504
1.161

In [10]:
# list to df
matrix_test = pd.DataFrame.from_records(lst_test)
matrix_test.set_axis(test_tables, axis=1, inplace=True)
matrix_test.rename(test_tables, inplace=True)
matrix_test.head()

,A,A+B,A1,A2,A3,A4,B,B1,C,D
A,0.000000,1.273089,0.904099,1.164702,1.074260,0.983403,0.951965,1.284972,0.885910,1.431642
A+B,1.273089,0.000000,0.948547,1.162887,0.980951,0.946548,0.946897,1.033721,1.035827,0.586491
A1,0.904099,0.948547,0.000000,1.034256,1.067677,1.140475,1.128183,1.057199,1.009189,1.210363
A2,1.164702,1.162887,1.034256,0.000000,1.118656,1.044462,1.210030,1.083776,1.023567,0.863789
A3,1.074260,0.980951,1.067677,1.118656,0.000000,0.944737,1.013769,0.873870,0.867387,0.990437


Distance Tree for Test Data

In [11]:
dm = DistanceMatrix(matrix_test, test_tables)
tree = nj(dm)
str_tree = tree.ascii_art()

text_file = open("result/tree_test.txt", "w")
text_file.write(str_tree)
text_file.close()